In [3]:
# Execute this cell to install medmnist if not present
!pip install medmnist

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/12.9 MB 8.4 MB/s eta 0:00:02
   --------- ------------------------------ 3.1/12.9 MB 8.0 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.9 MB 8.2 MB/s eta 0:00:01
   --------------------- ------------------ 6.8/12.9 MB 8.4 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.9 MB 9.1 MB/s eta 0:00:01
   ----------------------------------- ---- 11.5/12.9 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 9.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ------------- -------------------------- 2.1/6.2 MB 10.7 MB/s eta 0:00:01
   --------------------------- ------------ 4.2/6.2 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 10.0 MB/s

In [8]:
import numpy as np
from data import load_data


train_images, test_images, train_labels, test_labels = load_data()

# Checking dimemsions
N_tr, H_tr, W_tr, C_tr = train_images.shape
N_te, H_te, W_te, C_te = test_images.shape

print(f'Train images shape -- N_tr = {N_tr}, H_tr = {H_tr}, W_tr = {W_tr}, C_tr = {C_tr}')
print(f'Test images shape  -- N_te = {N_te}, H_te = {H_te}, W_te = {W_te}, C_te = {C_te}')

# Loading data to examine classes
extracted_classes = np.unique(train_labels)
n_classes = len(extracted_classes)

class_counts = np.bincount(train_labels.astype(int))

print(f'Classes:')
for c in extracted_classes:
    print(f'{c}')
print(f'Number of classes: {n_classes}')
print(f'Class counts: {class_counts}')

Using downloaded and verified file: C:\Users\alexf\.medmnist\dermamnist.npz
Using downloaded and verified file: C:\Users\alexf\.medmnist\dermamnist.npz
Train images shape -- N_tr = 7007, H_tr = 28, W_tr = 28, C_tr = 3
Test images shape  -- N_te = 2005, H_te = 28, W_te = 28, C_te = 3
Classes:
0
1
2
3
4
5
6
Number of classes: 7
Class counts: [ 228  359  769   80  779 4693   99]


In [9]:
def preprocess_data_mlp(train_images, test_images, train_labels, test_labels):
    """
    Preprocesses image data for MLP training
    
    Performs the following operations:
    1. Flattens 28x28x3 images into 1D vectors (2352 features)
    2. Normalizes pixel values from [0,255] to [0,1] range
    3. Creates stratified train/validation split (80/20) maintaining class proportions
    4. Calculates normalized class weights for handling imbalanced dataset
    
    Args:
        train_images: Training images array of shape (N, 28, 28, 3)
        test_images:  Test images array of shape (N_test, 28, 28, 3)
        train_labels: Training labels array of shape (N,)
        test_labels:  Test labels array of shape (N_test,)
    
    Returns:
        train_set_datapoints:      Flattened, normalized training data (80% of original)
        train_set_labels:          Corresponding training labels
        validation_set_datapoints: Flattened, normalized validation data (20% of original)
        validation_set_labels:     Corresponding validation labels
        proc_test_images:          Flattened, normalized test data
        test_labels:               Original test labels (unchanged)
        normalised_weights:        Class weights for loss function (shape: n_classes,)
    """
    # Image dimensions
    N_tr, H_tr, W_tr, C_tr = train_images.shape
    N_te, H_te, W_te, C_te = test_images.shape
    
    # Image flattening
    proc_train_images = train_images.reshape(N_tr, -1).astype(np.float32)
    proc_test_images  = test_images.reshape(N_te, -1).astype(np.float32)
    
    # Normalisation (min-max scaling - prevents negative values, maps 0-255 to 0-1)
    proc_train_images = proc_train_images / 255.0
    proc_test_images  = proc_test_images  / 255.0
    
    # Get number of classes
    n_classes = len(np.unique(train_labels))
    
    # Training, Validation set construction
    # Take 20% from each class to preserve proportions (for validation)
    p_validation = 0.2
    
    # Use lists for collecting data
    train_images_list = []
    train_labels_list = []
    validation_images_list = []
    validation_labels_list = []
    
    for class_index in range(n_classes):
        # Retrieving datapoint indices in train_images with given class index
        class_indices = np.where(train_labels == class_index)[0]
        
        # Shuffle the indices
        np.random.shuffle(class_indices)
        
        # Calculate split point
        n_validation = int(len(class_indices) * p_validation)
        
        # Split indices
        validation_indices = class_indices[:n_validation]
        training_indices   = class_indices[n_validation:]
        
        # Append to lists
        validation_images_list.append(proc_train_images[validation_indices])
        validation_labels_list.append(train_labels[validation_indices])
        train_images_list.append(proc_train_images[training_indices])
        train_labels_list.append(train_labels[training_indices])
    
    # Concatenate all lists
    train_set_datapoints      = np.vstack(train_images_list)
    train_set_labels          = np.concatenate(train_labels_list)
    validation_set_datapoints = np.vstack(validation_images_list)
    validation_set_labels     = np.concatenate(validation_labels_list)
    
    # Class weight calculation (using the training set after split)
    class_counts       = np.bincount(train_set_labels.astype(int))
    total_samples      = len(train_set_labels)
    raw_weights        = total_samples / class_counts
    normalised_weights = raw_weights * (n_classes / np.sum(raw_weights))
    
    return train_set_datapoints, train_set_labels, validation_set_datapoints, validation_set_labels, proc_test_images, test_labels, normalised_weights